In [1]:
from zipline.pipeline import Pipeline
from zipline.component.research import run_pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

## Putting It All Together
Now that we've covered the basic components of the Pipeline API, let's construct a pipeline that we might want to use in an algorithm.

To start, let's first create a filter to narrow down the types of securities coming out of our pipeline. In this example, we will create a filter to select for securities that meet all of the following criteria:
- Is a primary share
- Is listed as a common stock
- Is not a [depositary receipt](http://www.investopedia.com/terms/d/depositaryreceipt.asp) (ADR/GDR)
- Is not trading [over-the-counter](http://www.investopedia.com/terms/o/otc.asp) (OTC)
- Is not [when-issued](http://www.investopedia.com/terms/w/wi.asp) (WI)
- Doesn't have a name indicating it's a [limited partnership](http://www.investopedia.com/terms/l/limitedpartnership.asp) (LP)
- Doesn't have a company reference entry indicating it's a LP
- Is not an [ETF](http://www.investopedia.com/terms/e/etf.asp) (has Morningstar fundamental data)


#### Why These Criteria?
Selecting for primary shares and common stock helps us to select only a single security for each company. In general, primary shares are a good representative asset of a company so we will select for these in our pipeline.

ADRs and GDRs are issuances in the US equity market for stocks that trade on other exchanges. Frequently, there is inherent risk associated with depositary receipts due to currency fluctuations so we exclude them from our pipeline.

OTC, WI, and LP securities are not tradeable with most brokers. As a result, we exclude them from our pipeline.

### Creating Our Pipeline
Let's create a filter for each criterion and combine them together to create a `tradeable_stocks` filter. First, we need to import the Morningstar `DataSet` as well as the `IsPrimaryShare` builtin filter.

In [2]:
from zipline.pipeline.data import Fundamentals
from zipline.component.filters import IsPrimaryShare, CNN

Now we can define our filters:

使用`CNN`过滤。如取前100位，使用`CNN(100)`。
<span class="girk">请注意，`CNN`过滤基准是近期成交额排位，而非总市值或者是流通市值。</span>

Note that when defining our filters, we used several `Classifier` methods that we haven't yet seen including `notnull`, `startswith`, `endswith`, and `matches`. Documentation on these methods is available [here](https://www.zipline.com/help#zipline_pipeline_classifiers_Classifier).

Next, let's create a filter for the top 30% of tradeable stocks by 20-day average dollar volume. We'll call this our `base_universe`.

In [3]:
base_universe = AverageDollarVolume(window_length=20, mask=IsPrimaryShare()).percentile_between(70, 100)

#### Built-in Base Universe

We have just defined our own base universe to select 'tradeable' securities with high dollar volume. However, zipline has two built-in filters that do something similar. The [Q500US](https://www.zipline.com/help#zipline_pipeline_filters_Q500US) and the [Q1500US](https://www.zipline.com/help#zipline_pipeline_filters_Q1500US) are built-in pipeline filters that select a group of 500 or 1500 tradeable, liquid stocks each day. Constituents of these groups are chosen at the start of each calendar month by selecting the top 'tradeable' stocks by 200-day average dollar volume, capped at 30% of equities allocated to any single sector (more detail on the selection criteria of these filters can be found [here](https://www.zipline.com/posts/the-q500us-and-q1500us)).

To simplify our pipeline, let's replace what we've already written for our `base_universe` with the `Q1500US` built-in filter. First, we need to import it.

Then, let's set our base_universe to the `Q1500US`.

In [4]:
base_universe = CNN(1500)

Now that we have a filter `base_universe` that we can use to select a subset of securities, let's focus on creating factors for this subset. For this example, let's create a pipeline for a mean reversion strategy. In this strategy, we'll look at the 10-day and 30-day moving averages (close price). Let's plan to open equally weighted long positions in the 75 securities with the least (most negative) percent difference and equally weighted short positions in the 75 with the greatest percent difference. To do this, let's create two moving average factors using our `base_universe` filter as a mask. Then let's combine them into a factor computing the percent difference.

In [5]:
# 10-day close price average.
mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=base_universe)

# 30-day close price average.
mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=base_universe)

percent_difference = (mean_10 - mean_30) / mean_30

Next, let's create filters for the top 75 and bottom 75 equities by `percent_difference`.

In [6]:
# Create a filter to select securities to short.
shorts = percent_difference.top(75)

# Create a filter to select securities to long.
longs = percent_difference.bottom(75)

Let's then combine `shorts` and `longs` to create a new filter that we can use as the screen of our pipeline:

In [7]:
securities_to_trade = (shorts | longs)

Since our earlier filters were used as masks as we built up to this final filter, when we use `securities_to_trade` as a screen, the output securities will meet the criteria outlined at the beginning of the lesson (primary shares, non-ETFs, etc.). They will also have high dollar volume.

Finally, let's instantiate our pipeline. Since we are planning on opening equally weighted long and short positions later, the only information that we actually need from our pipeline is which securities we want to trade (the pipeline index) and whether or not to open a long or a short position. Let's add our `longs` and `shorts` filters to our pipeline and set our screen to be `securities_to_trade`.

In [8]:
def make_pipeline():
    
    # Base universe filter.
    # base_universe = Q1500US()
    base_universe = CNN(1500)
    
    # 10-day close price average.
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=base_universe)

    # 30-day close price average.
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=base_universe)

    # Percent difference factor.
    percent_difference = (mean_10 - mean_30) / mean_30
    
    # Create a filter to select securities to short.
    shorts = percent_difference.top(75)

    # Create a filter to select securities to long.
    longs = percent_difference.bottom(75)
    
    # Filter for the securities that we want to trade.
    securities_to_trade = (shorts | longs)
    
    return Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts
        },
        screen=securities_to_trade
    )

Running this pipeline will result in a DataFrame with 150 rows and 2 columns each day. Each day, the columns will contain boolean values that we can use to decide whether we want to open a long or short position in each security.

In [9]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result

longs  shorts
2015-05-05 00:00:00+00:00 Equity(000065 [北方国际])   False    True
                          Equity(000066 [中国长城])    True   False
                          Equity(000078 [海王生物])   False    True
                          Equity(000156 [华数传媒])    True   False
                          Equity(000410 [*ST沈机])  False    True
                          Equity(000415 [渤海金控])   False    True
                          Equity(000516 [国际医学])   False    True
                          Equity(000517 [荣安地产])   False    True
                          Equity(000592 [平潭发展])    True   False
                          Equity(000681 [视觉中国])    True   False
                          Equity(000806 [银河生物])   False    True
                          Equity(000812 [陕西金叶])    True   False
                          Equity(000835 [长城动漫])    True   False
                          Equity(000851 [高鸿股份])    True   False
                          Equity(000859 [国风塑业])   False    True
                          Equity(000875 [吉电股份])   False    True
                          Equity(000893 [东凌国际])    True   False
                          Equity(000923 [河北宣工])   False    True
                          Equity(000930 [中粮生化])   False    True
                          Equity(000936 [华西股份])   False    True
                          Equity(000967 [盈峰环境])   False    True
                          Equity(000971 [高升控股])   False    True
                          Equity(002009 [天奇股份])    True   False
                          Equity(002031 [巨轮智能])    True   False
                          Equity(002044 [美年健康])   False    True
                          Equity(002070 [*ST众和])   True   False
                          Equity(002079 [苏州固锝])    True   False
                          Equity(002081 [金 螳 螂])   True   False
                          Equity(002084 [海鸥卫浴])    True   False
                          Equity(002095 [生 意 宝])   True   False
...                                                 ...     ...
                          Equity(600637 [东方明珠])   False    True
                          Equity(600654 [*ST中安])  False    True
                          Equity(600661 [新南洋])     True   False
                          Equity(600665 [天地源])    False    True
                          Equity(600682 [南京新百])   False    True
                          Equity(600688 [上海石化])   False    True
                          Equity(600749 [西藏旅游])   False    True
                          Equity(600794 [保税科技])    True   False
                          Equity(600804 [鹏博士])     True   False
                          Equity(600832 [东方明珠])   False    True
                          Equity(600880 [博瑞传播])    True   False
                          Equity(600978 [宜华生活])   False    True
                          Equity(601018 [宁波港])    False    True
                          Equity(601106 [*ST一重])  False    True
                          Equity(601179 [中国西电])   False    True
                          Equity(601299 [中国北车])   False    True
                          Equity(601388 [怡球资源])   False    True
                          Equity(601390 [中国中铁])   False    True
                          Equity(601600 [中国铝业])   False    True
                          Equity(601618 [中国中冶])   False    True
                          Equity(601669 [中国电建])   False    True
                          Equity(601718 [际华集团])   False    True
                          Equity(601727 [上海电气])   False    True
                          Equity(601766 [中国中车])   False    True
                          Equity(601866 [中远海发])   False    True
                          Equity(601877 [正泰电器])    True   False
                          Equity(601919 [中远海控])   False    True
                          Equity(601989 [中国重工])   False    True
                          Equity(603686 [龙马环卫])   False    True
                          Equity(603869 [北部湾旅])   False    True

[150 rows x 2 columns]

In the next lesson, we'll add this pipeline to an algorithm.